In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
flagD = 0
th = 30

ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)

# SBML model we want to import
sbml_file = 'BigModel_FULL_v4.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4] # 'BigModel_byparts_v1'
# Directory to which the generated model code is written
model_output_dir = model_name

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

kGsRead = pd.read_excel('kGeneMod.xlsx',header=0,index_col=0)
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
# # Make serum-fed population of random cells
numStocCells = 20
STIMin = [1.0,4.0,10.0,25.0] # GM cond
namexlsfile = 'Fig3F_20cells_'  

Vn = 1.75E-12 # get from model somehow
Vc = 5.25E-12 # get from model somehow
spdata0 = pd.read_excel('Species_v4.xlsx',header=0,index_col=0)
spdata = np.double(spdata0.values[:,1])
genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)

tout_all = np.arange(0,th*3600+1,30)
model.setFixedParameterById('k487',0.0) # kA67 == XIAP+Apop
model.setFixedParameterById('k489',0.0) # kA69 == XIAP+Smac

startTime = datetime.now()
print(startTime)
for ss in range(len(STIMin)): 
    for nn in range(numStocCells): 
        spdata2 = spdata.copy()
        spdata2[30] = STIMin[ss]
        xoutS_all = np.zeros(shape=(NSteps+1,len(spdata2)))
        xoutS_all[0,:] = spdata2
        xoutG_all = np.zeros(shape=(NSteps+1,len(genedata0)))
        xoutG_all[0,:] = genedata0
        model2 = model
        for qq in range(NSteps):
            genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                       kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
            for ii,kk in enumerate(kmRNAs):
                model2.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
            model2.setInitialStates(xoutS_all[qq,:])
            rdata = amici.runAmiciSimulation(model2, solver)  # Run simulation
            xoutS_all[qq+1,:] = rdata['x'][-1,:]
            xoutG_all[qq+1,:] = genedata
            if rdata['x'][-1,103] < rdata['x'][-1,105]:
                print('Apoptosis happened')
                break
        print([ss,nn])
        xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
        xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
        print(datetime.now() - startTime)
        condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model2.getStateIds()]) 
        condsSDF.to_excel(namexlsfile+'S_'+str(ss)+str(nn)+'.xlsx')    
        condsSDF = None
        condsGDF = pd.DataFrame(data=xoutG_all) 
        condsGDF.to_excel(namexlsfile+'G_'+str(ss)+str(nn)+'.xlsx')    
        condsGDF = None    
        print(datetime.now() - startTime)
print(datetime.now())

In [ ]:
currentDirectory = os.getcwd()
namexls = 'Fig3F_20cells_S_'

numStocCells = 5 #20
STIMin = [1.0,4.0,10.0,25.0] # GM cond

fig, axs = plt.subplots(4,1) #, figsize=(15, 6), facecolor='w', edgecolor='k')
axs = axs.ravel()
for aa in range(4):
    for cc in range(numStocCells):
        print([aa, cc])
        namer = currentDirectory+'\\Fig3F\\'+namexls+str(aa)+str(cc)+'.xlsx'
        condsDF = pd.read_excel(namer,header=0,index_col=0)
        condsDF = np.double(condsDF.values[:])
        condsDF = condsDF[~np.all(condsDF == 0, axis=1)]
        yy = np.array(condsDF[:,2])
        tt = (np.arange(0,len(condsDF))*30.0/3600.0)
        axs[aa].plot(tt, yy,linewidth=4)
    if aa==0:
        axs[aa].set_ylim([0, 20])
    else:
        axs[aa].set_ylim([0, 1500])
    axs[aa].set_xlim([0, 30])
#     axs[aa].set_title('DamageDSB = '+str(STIMin[aa]))
plt.savefig('Fig3F_1.png')